In [3]:
import os
import pdfplumber
import glob
import re
import pandas as pd
import numpy as np

### Extract affordable housing financing data from Ohio PDFs


In [4]:
#filepath to PDFs
filepath = 'Ohio 2019 Applications/'
# create a list of filepaths 
files = glob.glob(filepath+"*.pdf", recursive = True)

The pdf reader does not correctly extract tables from these documents, however it does pull data out in a relativley predictable pattern. Below I check for files that have the same structure - only 4 out of 121 do not, so instead of worry about the edge cases I store them in the list "broken files" and will deal with those manually.

In [109]:
for file in files:
    pdf = pdfplumber.open(file)
    #print(len(pdf.pages[0].extract_tables()[0]))
    df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
    if df.shape[0] > 40 and df.iloc[37].values[1] == 'Construction Financing Sources':
        continue
    else:
        print(file, df.shape)

Ohio 2019 Applications/MayflowerManor.pdf (56, 14)
Ohio 2019 Applications/BerwynEastPlace.pdf (25, 9)
Ohio 2019 Applications/AppleseedSeniorApartments.pdf (35, 14)
Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf (35, 13)


In [111]:
broken_files = ["Ohio 2019 Applications/MayflowerManor.pdf",
"Ohio 2019 Applications/BerwynEastPlace.pdf",
"Ohio 2019 Applications/AppleseedSeniorApartments.pdf",
"Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf"]

files = list(set(files) - set(broken_files)) #remove broken files from master set of files 

In [126]:
def scrape_ohio(files):
    """
    Inputs:
        Files (list): List of full filepaths to the PDFs for Virginia. 
    This file notices that while the extract table function of the PDF reader does not exactly work
    in capturing the correct tables, the data follows a similar pattern and so it slices into 
    the files and grabs the relevent info from "Construction Financing Sources" 
    and "Permenant Financing Sources"
    """
    money = []
    for file in files:
        pdf = pdfplumber.open(file)
        df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
        money_type = df.loc[37,1] #construction
        file_name = re.split("/", file)[-1] #regex to pull the filename from the filepath
        file_name = file_name[0:-4]

        for row in df[38:][1]:
            if 'TOTAL' in row:
                break
            else:
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])
        money_type = df.loc[37,6] #permanent
        for row in df[38:][6]:
            if 'TOTAL' in row:
                break
            else:
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])

    money_df = pd.DataFrame(money, columns=['Source',"Amount of Funds","Funding Type","File Name"])
    money_df = money_df[~money_df["Amount of Funds"].str.contains("-")] #drop the blanks 
    
    return money_df 

In [136]:
#drop duplicates 
df1 = df[~df["File Name"].str.contains("(1)")]
df = pd.read_csv("Ohio_scraped_financing_info.csv")

In [144]:
#write to csv
df1.to_csv("Ohio_scraped_financing_info.csv")